In [3]:
from gymnasium.utils.play import play
import gymnasium as gym


def play_env():
    env = gym.make("ALE/BankHeist-v5", render_mode="rgb_array")
    env.metadata['render_fps'] = 15
    play(env, zoom=3)

play_env()